## Script to delete round-robined data across FileDB

TODO turn this into a Python file or utils function

In [1]:
dest_folder_name = "sabl2048a"
cube_side = 4 # 64 total 512-cubes - 4x4x4
prod_or_backup = 'back'

timestep_range = range(5)

In [2]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

[Errno 2] No such file or directory: '/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing'
/home/idies/workspace/Storage/ariel4/persistent/zarrify-across-network/src/utils


In [3]:
import write_utils
import os

In [4]:
%cd /home/idies/workspace/turb/

/home/idies/workspace/turb


In [5]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=write_utils.list_fileDB_folders()
folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/',
 '/home/idies/workspace/turb/data02_01/zarr/',
 '/home/idies/workspace/turb/data03_01/zarr/',
 '/home/idies/workspace/turb/data04_01/zarr/',
 '/home/idies/workspace/turb/data05_01/zarr/']

In [6]:
# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/") # This is already created

In [7]:
for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_" + prod_or_backup

folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/sabl2048a_01_back',
 '/home/idies/workspace/turb/data02_01/zarr/sabl2048a_02_back',
 '/home/idies/workspace/turb/data03_01/zarr/sabl2048a_03_back',
 '/home/idies/workspace/turb/data04_01/zarr/sabl2048a_04_back',
 '/home/idies/workspace/turb/data05_01/zarr/sabl2048a_05_back']

In [8]:
folders_to_delete = folders

In [24]:
# ordering = write_utils.morton_order_cube(cube_side=4)
ordering = write_utils.node_assignment(4)

folders_to_delete = []
num_chunks = cube_side

for t in timestep_range:
    for i in range(num_chunks):
        for j in range(num_chunks):
            for k in range(num_chunks):
                # Put e.g. z-order=4 to filedb05_01
                filedb_index = ordering[i][j][k] % len(folders) # Loop around
                # current_array = outer_dim[i][j][k]

#                 folders_to_delete.append(os.path.join(folders[filedb_index], dest_folder_name + str(ordering[i][j][k]).zfill(2) + "_" + str(t).zfill(3) + ".zarr"))
                folders_to_delete.append(os.path.join(folders[filedb_index - 1], dest_folder_name + str(ordering[i][j][k]).zfill(2) + "_" + str(t).zfill(3) + ".zarr"))

folders_to_delete[:5]

['/home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_000.zarr',
 '/home/idies/workspace/turb/data02_01/zarr/sabl2048b_02_prod/sabl2048b02_000.zarr',
 '/home/idies/workspace/turb/data03_01/zarr/sabl2048b_03_prod/sabl2048b03_000.zarr',
 '/home/idies/workspace/turb/data04_01/zarr/sabl2048b_04_prod/sabl2048b04_000.zarr',
 '/home/idies/workspace/turb/data05_01/zarr/sabl2048b_05_prod/sabl2048b05_000.zarr']

In [9]:
import concurrent.futures

In [10]:
# ChatGPT
def delete_folder(folder):
    try:
        os.system(f'rm -rf {folder}')
#         print(f"Deleted folder: {folder}")
    except Exception as e:
        print(f"Error deleting folder: {folder}")
        print(e)

In [11]:
# Create a ThreadPoolExecutor with the desired number of worker threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the delete_folder function for each folder to the executor
    future_to_folder = {executor.submit(delete_folder, folder): folder for folder in folders_to_delete}

    # Wait for the deletion tasks to complete
    for future in concurrent.futures.as_completed(future_to_folder):
        folder = future_to_folder[future]
        try:
            future.result()
        except Exception as e:
            print(f"Error deleting folder: {folder}")
            print(e)